# Task 1 - Big Data Analysis using PySpark 

This notebook demonstrates big data analysis using **PySpark** on the **2013 NYC Yellow Taxi Trip Dataset**, a real-world large dataset.


In [ ]:
#Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NYC Taxi Data Analysis").getOrCreate()

In [ ]:
#Reading the CSV file
df = spark.read.csv("yellow_tripdata_example.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+------------------+--------------

In [9]:
# Basic statistics
print(f"Total records: {df.count()}")
df.describe(["trip_distance", "fare_amount", "total_amount"]).show()

Total records: 22
+-------+-----------------+------------------+------------------+
|summary|    trip_distance|       fare_amount|      total_amount|
+-------+-----------------+------------------+------------------+
|  count|               22|                22|                22|
|   mean|3.222272727272728|12.818181818181818|14.795909090909095|
| stddev|2.562470779054171| 7.272821969080461|  7.42788802842171|
|    min|             0.01|               2.5|               3.8|
|    max|            10.54|              33.0|              34.3|
+-------+-----------------+------------------+------------------+



In [10]:
# Most common pickup locations
df.groupBy("pickup_longitude", "pickup_latitude") \
  .count().orderBy("count", ascending=False).show(5)

+------------------+------------------+-----+
|  pickup_longitude|   pickup_latitude|count|
+------------------+------------------+-----+
|-73.96932983398438|  40.7635383605957|    1|
|-74.00614929199219| 40.74491882324219|    1|
|-73.94715118408203|40.791046142578125|    1|
|-73.99037170410156| 40.73469543457031|    1|
|-73.99834442138672| 40.72389602661133|    1|
+------------------+------------------+-----+
only showing top 5 rows


In [11]:
# Average trip distance per passenger count
df.groupBy("passenger_count").avg("trip_distance").orderBy("passenger_count").show()

+---------------+------------------+
|passenger_count|avg(trip_distance)|
+---------------+------------------+
|              1| 3.124615384615385|
|              2|             4.234|
|              3|              1.76|
|              4|              0.01|
|              5|               4.9|
|              6|              2.43|
+---------------+------------------+



In [12]:
# Filter out zero/negative fares
filtered_df = df.filter((df.total_amount > 0) & (df.fare_amount > 0))
filtered_df.select("fare_amount", "trip_distance", "passenger_count").show(5)

+-----------+-------------+---------------+
|fare_amount|trip_distance|passenger_count|
+-----------+-------------+---------------+
|        7.5|          1.1|              2|
|       18.0|          4.9|              5|
|       33.0|        10.54|              1|
|       16.5|         4.75|              1|
|        8.0|         1.76|              3|
+-----------+-------------+---------------+
only showing top 5 rows
